## SRJ - New idea

c1 |s1> + c2|s2>

## Original adapt code

## Single det.

In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []
operator_check = []  # To keep track of the highest operator values
alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def s1(params, ashs, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ashs):
            #print('The value of i', i)
            if len(ashs[i]) == 4:
                #print('The ash excitation now is', ashs[i])
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.expval(H)
    
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def s1state(hf_state, ashs, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Params in s1state are', params)
        #print('Ashs in s1state are', ashs)
        for i, excitations in enumerate(ashs):
            if len(ashs[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.state()

    def cost(params):
        energy = s1(params, ashs, hf_state, H)
        return energy
    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy
    
    @qml.qnode(dev)
    def s2(hf_state, latest_ash, params2, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
        for i in range(len(latest_ash)):
            #print('The value of i in s2 ', i)  
            if len(latest_ash[i]) == 4:
                #print('The latest ash excitation in s2 state in E calc is', latest_ash[i])
                qml.DoubleExcitation(params2[i], wires = latest_ash[i])  # Assuming latest_ash is a list of lists, each containing the wires for the excitation
            elif len(latest_ash[i]) == 2:
                qml.SingleExcitation(params2[i], wires = latest_ash[i]) 
        return qml.expval(H)

    @qml.qnode(dev1)
    def s2state(hf_state, ashs, params2):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i in range(len(latest_ash)):
            #print('The value of i', i)
            if len(latest_ash[i]) == 4:
                #print('The latest ash excitation in s2 state is', latest_ash[i])
                #print('params2 is', params2[i])
                qml.DoubleExcitation(params2[i], wires = latest_ash[i]) 
            elif len(latest_ash[i]) == 2:
                qml.SingleExcitation(params2[i], wires = latest_ash[i])
        return qml.state()
    dev2 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
    @qml.qnode(dev2)
    def measure(state):
        qml.StatePrep(state, wires=range(qubits))
        return qml.counts()
    
    def global_cost(params, params2, alpha, beta):
        s1o = alpha * s1state(hf_state, ashs, params)
        s2o = beta * s2state(hf_state, latest_ash, params2)
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))
        s1oc = qml.math.conj(s1o)
        s2oc = qml.math.conj(s2o)
        # Hamiltonian expectation calculations...
        term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
        #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
        term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
        #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
        term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
        #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
        term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
        numerator = term1 + term2 + term3 + term4
        #D1 = np.dot(s1oc, s1o).real
        D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
        #D2 = np.dot(s2oc, s2o).real
        D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
        #D3 = np.dot(s1oc, s2o).real
        D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
        #D4 = np.dot(s2oc, s1o).real
        D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
        denominator = D1 + D2 + D3 + D4
        return numerator / denominator

    def global_cost_flat(x):
        params = x[:len(ashs)]  # 0,1
        params2 = x[len(ashs):len(ashs) + 1] #2
        alpha = x[len(ashs)+1] #3
        beta = x[len(ashs)+2]  #4
        return global_cost(params, params2, alpha, beta)

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            if i not in operator_check:
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        operator_check.append(max_operator)
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        ashs = ash_excitation
        print('Len of ashs:', len(ashs))
        alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Exc. going to prepare state1',ashs)
        params = np.zeros(len(ashs), requires_grad=True)    
        #params = np.append(params, 0.0)  #Parameters initialization



        s1ostate = alpha * s1state(hf_state, ashs, params)
        print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

        #s2 state preparation
        latest_ash = [ashs[-1]]
        print("Latest excitation:", latest_ash)
        print('Length of latest excitation:', len(latest_ash))
        params2 = qml.numpy.array([np.pi/4], requires_grad=True)  # Initial parameters for the excitations
        beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Params2 are', params2)

        
        s2ostate = beta * s2state(hf_state, latest_ash, params2)
        print(qml.draw(s2state, max_length=100)(hf_state,latest_ash,params2))
        x0 = np.concatenate([params, params2, np.array([alpha]), np.array([beta])])

        print(x0)




        result = minimize(global_cost_flat, x0, method='BFGS')
        print(result)
        energies.append(result.fun)  # Appending the energy to the energies list
        params = result.x[:len(ashs)]  # Extracting the first part of the result as params
        params2 = result.x[len(ashs):len(ashs) + 1]  # Extracting the second part of the result as params2
        print('Params1 after optimization:', params)
        print('Params2 after optimization:', params2)
        alpha = result.x[len(ashs) + 1]  # Extracting the alpha parameter
        beta = result.x[len(ashs) + 2]  # Extracting the beta parameter
        print('Alpha after optimization:', alpha)
        print('Beta after optimization:', beta)

        #final_state = alpha * s1state(hf_state, ashs, params) + beta * s2state(hf_state, latest_ash, params2)
        #final_state = final_state / np.linalg.norm(final_state)

        final_state = s1state(hf_state, ashs, params)
        print("Norm after manual normalization:", np.linalg.norm(final_state)) 
        print('counts after the final state with optimization is', measure(final_state))
        states.append(final_state)  # Appending the new state to the states list
        
    return params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=25) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
The adapt iteration now is 0
The highest operator value is 0.3100451588582031 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 1
Exc. going to prepare state1 [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[2, 3, 6, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X───────────┤  State
2: ──X─╭G²(0.79)─┤  State
3: ──X─├G²(0.79)─┤  State
6: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successf

In [4]:
energies
old_grad

[0.3100451588582031,
 0.34678322371546566,
 0.38589301231067186,
 0.1810372345005346,
 0.11394892475060205,
 0.11979485988740776,
 0.044175187120723804,
 0.034612142891874524,
 0.041368871148446736,
 0.019715325428236515,
 0.04260329824431383,
 0.10382203151933021,
 0.01005024199340257,
 0.0006764585287712772,
 2.710502764656108e-16,
 6.187791355064812e-07,
 1.5140447842136072e-06,
 1.816279910176091e-06,
 2.843093662486155e-07,
 1.0285746008151453e-05,
 0.0025328875123424577,
 1.7079954995990637e-05,
 0.00266637440341904,
 7.308897263512076e-06,
 0.002577408560250764]

## Single det. case

In [2]:
print(energies)
print(old_grad)

[-1.430083670662174, -1.56946038355194, -1.7440626583933085, -1.8660667755314557, -1.8758503116042369, -1.8835192725160552, -1.8832018741837144, -1.8830798945612264, -1.883488000610656, -1.8848742852942832, -1.8849779254284131, -1.8849115567829753, -1.8849892293543276, -1.8850812589466992]
[0.3100451588582031, 0.34678322371546566, 0.38589301231067186, 0.18103723450053458, 0.11394892475060205, 0.11979485988740776, 0.044175187120723804, 0.034612142891874524, 0.041368871148446736, 0.019715325428236515, 0.04260329824431382, 0.10382203151933021, 0.01005024199340257, 0.0006764585287712785]


## This below data is for full state alpha|s1> + beta |s2>. Here opertors are repeating. Try wiht operator chcks

In [ ]:
energies


[-1.430083670662174,
 -1.56946038355194,
 -1.7440626583933085,
 -1.8660667755314557,
 -1.8758503116037928,
 -1.8746695587055973,
 -1.883275948278045,
 -1.8830178008309917,
 -1.8830178006420695,
 -1.8830178009885943,
 -1.8830178009444953,
 -1.8830178008857008,
 -1.8830178009910523,
 -1.883017801022203]

In [ ]:
old_grad

[0.3100451588582031,
 0.36625718143639074,
 0.4230543672527128,
 0.3002278044154616,
 0.11844160289748043,
 0.23335149443546266,
 0.10110207018250975,
 0.23865487130600274,
 0.2926406606160391,
 0.28728097555361054,
 0.30514306332920355,
 0.27952422155544554,
 0.297341784040302,
 0.3006919302930441]

## Data with repetitive operators 

In [2]:
# Check Case2-rep.py in Talon

In [ ]:
#Output is 
Changing to the working directory: /home/s.poyyapakkam/NH3/Aqsceom/GCIM

This is case 2 using Adapt like Gradients - s2 state is prepared with the latest excitation- Repetitive of operators
H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.332568888066893
Total excitations are 26
The adapt iteration now is 0
The expectation value of a⁺(0) a(4) is 9.258164662684365e-08
The expectation value of a⁺(0) a(6) is 0.0
The expectation value of a⁺(1) a(5) is 9.258164661296586e-08
The expectation value of a⁺(1) a(7) is 0.0
The expectation value of a⁺(2) a(4) is 0.0
The expectation value of a⁺(2) a(6) is 1.1698954668903916e-07
The expectation value of a⁺(3) a(5) is 0.0
The expectation value of a⁺(3) a(7) is 1.1698954668903916e-07
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.30574752590560217
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 0.0
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 0.0
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.28410686098084653
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.030544625693075148
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 0.0
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.3078190525120221
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.2772744268189469
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 0.0
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 0.0
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.2772744268189469
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.3078190525120221
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 0.0
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.030544625693075148
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.30057780517222726
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 0.0
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 0.0
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.3100451588582134
The highest operator value is 0.3100451588582134 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 1
Exc. going to prepare state1 [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[2, 3, 6, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X───────────┤  State
2: ──X─╭G²(0.79)─┤  State
3: ──X─├G²(0.79)─┤  State
6: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.4300836706628741
        x: [0.48841145+0.j  1.29888568+0.j  0.26021228+0.8j 0.9240021 +0.8j]
      nit: 4
      jac: [-1.654e-06 -6.169e-06  4.157e-06 -1.207e-06]
 hess_inv: [[ 3.155e+00  2.233e+00 -1.476e+00  1.428e+00]
            [ 2.233e+00  3.289e+00 -1.512e+00  1.480e+00]
            [-1.476e+00 -1.512e+00  1.999e+00 -9.781e-01]
            [ 1.428e+00  1.480e+00 -9.781e-01  1.947e+00]]
     nfev: 41
     njev: 7
Params1 after optimization: [0.48841145+0.j]
Params2 after optimization: [1.29888568+0.j]
Alpha after optimization: (0.2602122846058762+0.8j)
Beta after optimization: (0.9240021036508834+0.8j)
Norm after manual normalization: 0.9999999999999976
counts after the final state with optimization is {np.str_('11000011'): np.int64(60), np.str_('11110000'): np.int64(940)}
The adapt iteration now is 1
The expectation value of a⁺(0) a(4) is 0.004201950084305661
The expectation value of a⁺(0) a(6) is 0.0
The expectation value of a⁺(1) a(5) is 0.00420195008430557
The expectation value of a⁺(1) a(7) is 0.0
The expectation value of a⁺(2) a(4) is 0.0
The expectation value of a⁺(2) a(6) is 0.025993977948838475
The expectation value of a⁺(3) a(5) is 0.0
The expectation value of a⁺(3) a(7) is 0.025993977948838527
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.30574752590560067
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 0.0
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 0.0
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.18279986317025282
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.04345901516203324
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 0.0
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.30332420254213205
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.3467832177041654
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 0.0
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 0.0
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.3467832177041655
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.30332420254213216
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 0.0
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.04345901516203324
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.19619365003151543
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 0.0
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 0.0
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.20391304375356745
The highest operator value is 0.3467832177041655 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 2
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7]]
0: ──X───────────────────────────────────────────────────────────────────┤  State
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)─┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─│────────────────────────────────┤  State
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.78539816+0.j  0.6       +0.8j
 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.5694603835508567
        x: [ 0.9541298 +0.j  -1.21708543+0.j  -0.40086592+0.j
             1.38628365+0.8j -0.21060752+0.8j]
      nit: 8
      jac: [-8.494e-07 -3.874e-07  7.451e-08  2.980e-08  2.682e-07]
 hess_inv: [[ 2.141e+00  3.251e-01 ...  4.957e-01 -1.484e-01]
            [ 3.251e-01  3.494e+00 ... -9.829e-01  7.069e-01]
            ...
            [ 4.957e-01 -9.829e-01 ...  1.647e+00 -4.798e-01]
            [-1.484e-01  7.069e-01 ... -4.798e-01  1.209e+00]]
     nfev: 97
     njev: 14
Params1 after optimization: [ 0.9541298 +0.j -1.21708543+0.j]
Params2 after optimization: [-0.40086592+0.j]
Alpha after optimization: (1.3862836518456627+0.8j)
Beta after optimization: (-0.2106075241737971+0.8j)
Norm after manual normalization: 0.9999999999999947
counts after the final state with optimization is {np.str_('10011001'): np.int64(226), np.str_('11000011'): np.int64(212), np.str_('11110000'): np.int64(562)}
The adapt iteration now is 2
The expectation value of a⁺(0) a(4) is 0.0292985266059273
The expectation value of a⁺(0) a(6) is 1.7719668174087812e-17
The expectation value of a⁺(1) a(5) is 0.06473191398633088
The expectation value of a⁺(1) a(7) is 2.1282866375151417e-17
The expectation value of a⁺(2) a(4) is 1.9956617541069246e-17
The expectation value of a⁺(2) a(6) is 0.07590661930526435
The expectation value of a⁺(3) a(5) is 2.227306263088395e-17
The expectation value of a⁺(3) a(7) is 0.08377350953701324
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.426790864300441
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 9.928303674371614e-19
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 9.212963602037588e-19
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.06271551897727454
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.01836934372745008
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 3.940163790942507e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.04915877369121515
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.41238959913032563
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 2.7984891478012097e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 7.76043945930212e-19
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.04648317975961212
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.045823053328085596
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 9.470985371233142e-19
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.024466109641306997
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.06844974675339749
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 9.379006734143468e-19
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 4.478637390024377e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.07023822147373462
The highest operator value is 0.426790864300441 for operator a⁺(0) a⁺(1) a(4) a(5)
Highest gradient excitation is [0, 1, 4, 5]
Len of ashs: 3
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────┤  State
──│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─┤  State
───────────────────────────────────┤  State
───────────────────────────────────┤  State
Latest excitation: [[0, 1, 4, 5]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X─╭G²(0.79)─┤  State
1: ──X─├G²(0.79)─┤  State
2: ──X─│─────────┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
5: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.78539816+0.j
 0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.7440626583927687
        x: [ 1.45500438+0.j  -1.93320707+0.j   1.5181493 +0.j
             1.26753243+0.j   1.26796326+0.8j  0.41018509+0.8j]
      nit: 21
      jac: [-1.311e-06  4.321e-07  9.686e-07  3.725e-07 -3.576e-07
             1.162e-06]
 hess_inv: [[ 7.812e+00 -8.473e+00 ... -2.011e+00  4.609e+00]
            [-8.473e+00  2.360e+01 ...  3.808e+00 -1.022e+01]
            ...
            [-2.011e+00  3.808e+00 ...  2.748e+00 -2.678e+00]
            [ 4.609e+00 -1.022e+01 ... -2.678e+00  6.532e+00]]
     nfev: 191
     njev: 24
Params1 after optimization: [ 1.45500438+0.j -1.93320707+0.j  1.5181493 +0.j]
Params2 after optimization: [1.26753243+0.j]
Alpha after optimization: (1.2679632601528998+0.8j)
Beta after optimization: (0.4101850910806996+0.8j)
Norm after manual normalization: 0.9999999999999906
counts after the final state with optimization is {np.str_('00001111'): np.int64(231), np.str_('00111100'): np.int64(72), np.str_('10011001'): np.int64(381), np.str_('11000011'): np.int64(235), np.str_('11110000'): np.int64(81)}
The adapt iteration now is 3
The expectation value of a⁺(0) a(4) is 0.10423762217310359
The expectation value of a⁺(0) a(6) is 4.608638970649938e-18
The expectation value of a⁺(1) a(5) is 0.10570116298071108
The expectation value of a⁺(1) a(7) is 1.122838846063495e-17
The expectation value of a⁺(2) a(4) is 1.8715023659188084e-17
The expectation value of a⁺(2) a(6) is 0.11068862191584913
The expectation value of a⁺(3) a(5) is 6.766996461423773e-19
The expectation value of a⁺(3) a(7) is 0.09867445087912437
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.004138573688703441
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 1.4112287863491762e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.4252619897903937e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.002373890474326795
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.0014210297826920733
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.1856853195126874e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.006205339514128905
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.18103714704511886
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.7862100376599106e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 1.0530792146203081e-20
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.36630045661318256
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.01044523378198246
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 1.3909101668868298e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.009170373263737121
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.0008296731505172284
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 5.124577438475695e-20
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.2123938700431654e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.1789426768000871
The highest operator value is 0.36630045661318256 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 4
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
───────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667755901288
        x: [ 1.18475182+0.j  -2.11486667+0.j   1.88278171+0.j
             1.65457045+0.j   0.11811936+0.j   1.41311473+0.8j
            -0.46961998+0.8j]
      nit: 21
      jac: [-1.878e-06 -1.162e-06 -1.073e-06  1.505e-06  9.090e-07
            -5.215e-07 -1.356e-06]
 hess_inv: [[ 3.398e+00  4.765e-01 ...  1.516e+00  1.553e+00]
            [ 4.765e-01  3.415e+00 ... -3.374e-01  9.647e-01]
            ...
            [ 1.516e+00 -3.374e-01 ...  2.178e+00  8.563e-01]
            [ 1.553e+00  9.647e-01 ...  8.563e-01  2.239e+00]]
     nfev: 242
     njev: 27
Params1 after optimization: [ 1.18475182+0.j -2.11486667+0.j  1.88278171+0.j  1.65457045+0.j]
Params2 after optimization: [0.11811936+0.j]
Alpha after optimization: (1.4131147262542212+0.8j)
Beta after optimization: (-0.46961998399230165+0.8j)
Norm after manual normalization: 0.9999999999999869
counts after the final state with optimization is {np.str_('00001111'): np.int64(94), np.str_('00111100'): np.int64(114), np.str_('01100110'): np.int64(109), np.str_('10011001'): np.int64(107), np.str_('11000011'): np.int64(112), np.str_('11110000'): np.int64(464)}
The adapt iteration now is 4
The expectation value of a⁺(0) a(4) is 0.10448457336502387
The expectation value of a⁺(0) a(6) is 2.7132577905463476e-18
The expectation value of a⁺(1) a(5) is 0.10403746318984566
The expectation value of a⁺(1) a(7) is 4.298211336073369e-17
The expectation value of a⁺(2) a(4) is 3.4545781129489655e-17
The expectation value of a⁺(2) a(6) is 0.10406549848883889
The expectation value of a⁺(3) a(5) is 1.1543466888101767e-17
The expectation value of a⁺(3) a(7) is 0.10362666745982116
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.15280690083616447
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 7.495107312877064e-19
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.2037673619064022e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.03900615032603263
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.006577251776692228
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.8579693647054466e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.04264828261476891
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.14579696113970214
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.2118703861683708e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 7.369322214290176e-19
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.1637674731049467
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.04282502257843842
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 7.68499833733965e-19
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.006254220446480612
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.045414993797350915
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 6.005835949463644e-19
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.8765813265352053e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.15397982611160027
The highest operator value is 0.1637674731049467 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 5
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)───────────────────────────────────────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──│────────────────────────────────│────────────────────────────────│────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤

   State
   State
   State
   State
   State
   State
   State
   State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756515545
        x: [ 1.19484427+0.j  -2.10253855+0.j   1.88279196+0.j
             0.82727463+0.j   0.82727498+0.j   0.16361412+0.j
             1.43735989+0.8j -0.47099067+0.8j]
      nit: 21
      jac: [ 2.533e-06  7.302e-07 -1.371e-06  2.891e-06  2.876e-06
            -7.898e-07  1.937e-07  6.258e-07]
 hess_inv: [[ 3.507e+00  6.994e-01 ...  1.693e+00  1.671e+00]
            [ 6.994e-01  3.811e+00 ... -1.080e-01  1.081e+00]
            ...
            [ 1.693e+00 -1.080e-01 ...  2.406e+00  1.013e+00]
            [ 1.671e+00  1.081e+00 ...  1.013e+00  2.139e+00]]
     nfev: 279
     njev: 28
Params1 after optimization: [ 1.19484427+0.j -2.10253855+0.j  1.88279196+0.j  0.82727463+0.j
  0.82727498+0.j]
Params2 after optimization: [0.16361412+0.j]
Alpha after optimization: (1.4373598877872567+0.8j)
Beta after optimization: (-0.4709906676657105+0.8j)
Norm after manual normalization: 0.999999999999985
counts after the final state with optimization is {np.str_('00001111'): np.int64(90), np.str_('00111100'): np.int64(106), np.str_('01100110'): np.int64(113), np.str_('10011001'): np.int64(107), np.str_('11000011'): np.int64(112), np.str_('11110000'): np.int64(472)}
The adapt iteration now is 5
The expectation value of a⁺(0) a(4) is 0.10502573589913099
The expectation value of a⁺(0) a(6) is 1.0224263117014395e-18
The expectation value of a⁺(1) a(5) is 0.10442276857563207
The expectation value of a⁺(1) a(7) is 1.3337972738276912e-16
The expectation value of a⁺(2) a(4) is 1.1553028193482504e-16
The expectation value of a⁺(2) a(6) is 0.1046091290521331
The expectation value of a⁺(3) a(5) is 3.036657782959744e-17
The expectation value of a⁺(3) a(7) is 0.104017291366718
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.14979284609843074
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 7.852739422346695e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.2703598120486174e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.03774670762911153
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.0057278486819946
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.1747288122434492e-17
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.04249130891262055
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.14104955079502846
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.2509676068480299e-17
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 1.1453782622557135e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.16562011704197205
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.04273257282009566
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 7.857712756821951e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.005286885589404916
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.044076318267465164
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 6.185559106006961e-19
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.1970232904824043e-17
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.15094044790642752
The highest operator value is 0.16562011704197205 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 6
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

───────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.78539816+0.j  0.6       +0.8j
 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667755524885
        x: [ 1.26964111+0.j  -2.00678262+0.j   1.88281267+0.j
             0.55151702+0.j   0.55151765+0.j   0.55151744+0.j
             0.54838215+0.j   1.86092539+0.8j -0.55512351+0.8j]
      nit: 22
      jac: [ 7.302e-06 -1.878e-06  4.143e-06  7.451e-08  2.980e-08
             1.043e-07 -2.533e-07 -1.699e-06 -5.677e-06]
 hess_inv: [[ 3.647e+00  7.905e-01 ...  3.879e-01  2.955e+00]
            [ 7.905e-01  3.788e+00 ... -9.065e-01  1.193e+00]
            ...
            [ 3.879e-01 -9.065e-01 ...  1.798e+00  5.939e-01]
            [ 2.955e+00  1.193e+00 ...  5.939e-01  5.077e+00]]
     nfev: 329
     njev: 30
Params1 after optimization: [ 1.26964111+0.j -2.00678262+0.j  1.88281267+0.j  0.55151702+0.j
  0.55151765+0.j  0.55151744+0.j]
Params2 after optimization: [0.54838215+0.j]
Alpha after optimization: (1.860925385422122+0.8j)
Beta after optimization: (-0.5551235072208888+0.8j)
Norm after manual normalization: 0.9999999999999805
counts after the final state with optimization is {np.str_('00001111'): np.int64(100), np.str_('00111100'): np.int64(122), np.str_('01100110'): np.int64(132), np.str_('10011001'): np.int64(76), np.str_('11000011'): np.int64(116), np.str_('11110000'): np.int64(454)}
The adapt iteration now is 6
The expectation value of a⁺(0) a(4) is 0.10870912340626304
The expectation value of a⁺(0) a(6) is 1.5998511139635022e-18
The expectation value of a⁺(1) a(5) is 0.10706288613692225
The expectation value of a⁺(1) a(7) is 7.760244456375852e-17
The expectation value of a⁺(2) a(4) is 6.898579490668024e-17
The expectation value of a⁺(2) a(6) is 0.1083142880537915
The expectation value of a⁺(3) a(5) is 5.0783302857556594e-18
The expectation value of a⁺(3) a(7) is 0.10669746268116839
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.12706835656641724
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 8.15654127461734e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.1556439198452708e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.02877757279219975
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 2.6913372518556737e-05
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 7.581346819724042e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.04087427527070993
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.10402749314948817
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.1162714968972389e-17
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 3.028205522203646e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.1788165646883157
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.041599869388787895
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 7.96508815436212e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.0013530969265386902
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.034492955298517855
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 3.140514270254655e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 7.734130738470235e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.12803993084581206
The highest operator value is 0.1788165646883157 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 7
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.78539816+0.j
 0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756599194
        x: [ 1.29234775+0.j  -1.97604802+0.j   1.88279377+0.j
             0.41363375+0.j   0.41363386+0.j   0.41363427+0.j
             0.41363393+0.j   0.68169594+0.j   1.36667927+0.8j
            -0.39844369+0.8j]
      nit: 24
      jac: [ 2.742e-06 -4.023e-07 -3.427e-07 -2.533e-07 -2.831e-07
            -2.682e-07 -2.533e-07 -3.427e-07 -4.619e-07 -1.356e-06]
 hess_inv: [[ 3.230e+00  2.162e-01 ...  9.551e-01  1.513e+00]
            [ 2.162e-01  4.143e+00 ... -5.145e-01  4.537e-01]
            ...
            [ 9.551e-01 -5.145e-01 ...  1.699e+00  8.810e-01]
            [ 1.513e+00  4.537e-01 ...  8.810e-01  2.201e+00]]
     nfev: 335
     njev: 28
Params1 after optimization: [ 1.29234775+0.j -1.97604802+0.j  1.88279377+0.j  0.41363375+0.j
  0.41363386+0.j  0.41363427+0.j  0.41363393+0.j]
Params2 after optimization: [0.68169594+0.j]
Alpha after optimization: (1.3666792705533282+0.8j)
Beta after optimization: (-0.3984436857342075+0.8j)
Norm after manual normalization: 0.9999999999999788
counts after the final state with optimization is {np.str_('00001111'): np.int64(118), np.str_('00111100'): np.int64(140), np.str_('01100110'): np.int64(114), np.str_('10011001'): np.int64(67), np.str_('11000011'): np.int64(113), np.str_('11110000'): np.int64(448)}
The adapt iteration now is 7
The expectation value of a⁺(0) a(4) is 0.10970587604946763
The expectation value of a⁺(0) a(6) is 1.75753863158137e-18
The expectation value of a⁺(1) a(5) is 0.10778535027319532
The expectation value of a⁺(1) a(7) is 3.255382455455326e-17
The expectation value of a⁺(2) a(4) is 5.0072998704761134e-17
The expectation value of a⁺(2) a(6) is 0.10931924847916256
The expectation value of a⁺(3) a(5) is 1.8514211877365844e-17
The expectation value of a⁺(3) a(7) is 0.1074326584030984
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.1200431895760507
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 1.261609436102143e-17
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.6961957108394407e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.026193708282261863
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.0015708143709550627
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.2379006174464273e-17
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.040214595776887886
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.09215473835809432
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.641405944411542e-17
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 2.0515762301980486e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.1825860536823775
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.04108869863819056
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 1.2372378943701428e-17
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.003168430462789235
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.031712827788926784
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 2.50843603586602e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.2536374866052376e-17
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.12095717120235636
The highest operator value is 0.1825860536823775 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 8
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

─────────────────────────────────────────────────────────────────────────────────────────────────────┤
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──│────────────────────────────────│────────────────────────────────│────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤

   State
   State
   State
   State
   State
   State
   State
   State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667755164622
        x: [ 1.35359108+0.j  -1.88873427+0.j   1.88276274+0.j
             0.33090561+0.j   0.33090626+0.j   0.33090557+0.j
             0.33090514+0.j   0.33090555+0.j   1.07491925+0.j
             1.44629882+0.8j -0.40459088+0.8j]
      nit: 23
      jac: [-6.065e-06  1.550e-06 -6.288e-06 -4.470e-07 -4.619e-07
            -4.470e-07 -4.172e-07 -4.321e-07  4.768e-07  1.445e-06
             5.275e-06]
 hess_inv: [[ 2.946e+00 -1.184e-01 ...  1.113e+00  9.900e-01]
            [-1.184e-01  4.210e+00 ... -1.088e+00 -1.503e-01]
            ...
            [ 1.113e+00 -1.088e+00 ...  2.374e+00  1.246e+00]
            [ 9.900e-01 -1.503e-01 ...  1.246e+00  2.008e+00]]
     nfev: 363
     njev: 28
Params1 after optimization: [ 1.35359108+0.j -1.88873427+0.j  1.88276274+0.j  0.33090561+0.j
  0.33090626+0.j  0.33090557+0.j  0.33090514+0.j  0.33090555+0.j]
Params2 after optimization: [1.07491925+0.j]
Alpha after optimization: (1.4462988150000788+0.8j)
Beta after optimization: (-0.40459087715307995+0.8j)
Norm after manual normalization: 0.9999999999999751
counts after the final state with optimization is {np.str_('00001111'): np.int64(110), np.str_('00111100'): np.int64(124), np.str_('01100110'): np.int64(146), np.str_('10011001'): np.int64(59), np.str_('11000011'): np.int64(137), np.str_('11110000'): np.int64(424)}
The adapt iteration now is 8
The expectation value of a⁺(0) a(4) is 0.11208219894936423
The expectation value of a⁺(0) a(6) is 1.4751236463610032e-17
The expectation value of a⁺(1) a(5) is 0.10952948080781862
The expectation value of a⁺(1) a(7) is 2.3510917204246738e-17
The expectation value of a⁺(2) a(4) is 5.429434740076694e-17
The expectation value of a⁺(2) a(6) is 0.11172117984015159
The expectation value of a⁺(3) a(5) is 5.684688372210169e-18
The expectation value of a⁺(3) a(7) is 0.10921190995493675
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.10084434678097304
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 2.5556935061565774e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 9.36604940983065e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.019597682730142545
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.005209820682750146
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.0969874916376507e-17
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.03801700955109467
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.05864548333280367
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 9.359302067652623e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 7.852254065178315e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.1921376697111764
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.039294231735059235
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 2.177440708572029e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.007544226396636612
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.02456494203664889
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 7.885210723157825e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.1217606418258124e-17
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.10160400219481086
The highest operator value is 0.1921376697111764 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 9
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

───────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.866066775450773
        x: [ 1.4277313 +0.j  -1.77304647+0.j   1.88278722+0.j
             0.27575766+0.j   0.27575885+0.j   0.27575806+0.j
             0.2757588 +0.j   0.27575908+0.j   0.27575707+0.j
             1.59022787+0.j   1.55357602+0.8j -0.43560404+0.8j]
      nit: 21
      jac: [ 9.745e-06 -3.725e-06 -3.681e-06  2.116e-06  2.176e-06
             2.161e-06  2.131e-06  2.190e-06  2.116e-06 -5.215e-07
             2.235e-07  9.984e-07]
 hess_inv: [[ 2.385e+00 -5.347e-01 ...  6.359e-01  6.253e-01]
            [-5.347e-01  4.269e+00 ... -1.092e+00 -1.019e+00]
            ...
            [ 6.359e-01 -1.092e+00 ...  1.791e+00  7.863e-01]
            [ 6.253e-01 -1.019e+00 ...  7.863e-01  3.087e+00]]
     nfev: 349
     njev: 25
Params1 after optimization: [ 1.4277313 +0.j -1.77304647+0.j  1.88278722+0.j  0.27575766+0.j
  0.27575885+0.j  0.27575806+0.j  0.2757588 +0.j  0.27575908+0.j
  0.27575707+0.j]
Params2 after optimization: [1.59022787+0.j]
Alpha after optimization: (1.5535760154351432+0.8j)
Beta after optimization: (-0.435604044726026+0.8j)
Norm after manual normalization: 0.9999999999999711
counts after the final state with optimization is {np.str_('00001111'): np.int64(129), np.str_('00111100'): np.int64(137), np.str_('01100110'): np.int64(148), np.str_('10011001'): np.int64(32), np.str_('11000011'): np.int64(146), np.str_('11110000'): np.int64(408)}
The adapt iteration now is 9
The expectation value of a⁺(0) a(4) is 0.11428363594529006
The expectation value of a⁺(0) a(6) is 9.885817744046473e-19
The expectation value of a⁺(1) a(5) is 0.11119829966263838
The expectation value of a⁺(1) a(7) is 1.0766960789178144e-16
The expectation value of a⁺(2) a(4) is 4.7545244501753914e-17
The expectation value of a⁺(2) a(6) is 0.11396158018336049
The expectation value of a⁺(3) a(5) is 1.7130502499670946e-17
The expectation value of a⁺(3) a(7) is 0.11092595296709559
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.07720921483572449
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 4.516507312200253e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 5.085039457976993e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.012448227454132342
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.008473653350931148
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 3.4086958414331593e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.03447807292207209
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.015160208839703482
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 4.787996853903197e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 1.81034469483093e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.20229430833407355
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.036245628803764654
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 4.507213533968686e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.011704252225301987
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.01670280158509092
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 1.6986497675550561e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 3.554311770922107e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.07775623558473294
The highest operator value is 0.20229430833407355 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 10
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.78539816+0.j  0.6       +0.8j
 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.866066775578696
        x: [ 1.39635674+0.j  -1.8234467 +0.j  ...  1.99174141+0.8j
            -0.5536352 +0.8j]
      nit: 25
      jac: [-4.753e-06 -5.960e-08 ... -1.013e-06 -3.606e-06]
 hess_inv: [[ 3.004e+00 -1.380e-01 ...  1.350e-02  1.665e+00]
            [-1.380e-01  4.056e+00 ... -1.858e+00 -4.651e-02]
            ...
            [ 1.350e-02 -1.858e+00 ...  1.890e+00 -7.824e-01]
            [ 1.665e+00 -4.651e-02 ... -7.824e-01  6.023e+00]]
     nfev: 494
     njev: 33
Params1 after optimization: [ 1.39635674+0.j -1.8234467 +0.j  1.88277976+0.j  0.23636148+0.j
  0.23636303+0.j  0.23636206+0.j  0.23636274+0.j  0.23636231+0.j
  0.23636196+0.j  0.23636276+0.j]
Params2 after optimization: [1.37003825+0.j]
Alpha after optimization: (1.9917414113502392+0.8j)
Beta after optimization: (-0.5536351987347539+0.8j)
Norm after manual normalization: 0.9999999999999699
counts after the final state with optimization is {np.str_('00001111'): np.int64(125), np.str_('00111100'): np.int64(145), np.str_('01100110'): np.int64(141), np.str_('10011001'): np.int64(40), np.str_('11000011'): np.int64(154), np.str_('11110000'): np.int64(395)}
The adapt iteration now is 10
The expectation value of a⁺(0) a(4) is 0.11344910785738638
The expectation value of a⁺(0) a(6) is 7.870382111546154e-20
The expectation value of a⁺(1) a(5) is 0.11055597127382222
The expectation value of a⁺(1) a(7) is 9.731793494866592e-17
The expectation value of a⁺(2) a(4) is 7.142291461532505e-17
The expectation value of a⁺(2) a(6) is 0.11310908455461191
The expectation value of a⁺(3) a(5) is 1.0778513705399672e-17
The expectation value of a⁺(3) a(7) is 0.11026343264559721
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.08724514755374049
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 3.1505448799966147e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 2.8093169240055553e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.015350639307045437
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.007256769891159832
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 7.499401098293624e-19
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.036098656787112805
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.03394984307599766
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 2.580703301478341e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 1.038797134437381e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.1982043729227405
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.037658705329428756
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 3.1033972856162415e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.010108103258874013
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.019912852980637503
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 9.867503057555589e-19
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 7.756600604967533e-19
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.08790069795734233
The highest operator value is 0.1982043729227405 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 11
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

─────────────────────────────────────────────────────────────────────────────────────────────────────┤
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──│────────────────────────────────│────────────────────────────────│────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤

   State
   State
   State
   State
   State
   State
   State
   State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.78539816+0.j
 0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756202018
        x: [ 1.53005727+0.j  -1.58917351+0.j  ...  2.1020775 +0.8j
            -0.65445733+0.8j]
      nit: 25
      jac: [-1.475e-06 -2.980e-08 ...  8.643e-07  2.757e-06]
 hess_inv: [[ 2.966e+00 -9.063e-01 ...  8.929e-01  6.681e-01]
            [-9.063e-01  5.146e+00 ... -3.046e+00 -2.234e+00]
            ...
            [ 8.929e-01 -3.046e+00 ...  3.516e+00  2.030e+00]
            [ 6.681e-01 -2.234e+00 ...  2.030e+00  5.899e+00]]
     nfev: 591
     njev: 37
Params1 after optimization: [ 1.53005727+0.j -1.58917351+0.j  1.88280885+0.j  0.20681533+0.j
  0.2068137 +0.j  0.2068162 +0.j  0.20681678+0.j  0.20681695+0.j
  0.20681694+0.j  0.20681497+0.j  0.20681662+0.j]
Params2 after optimization: [2.27620002+0.j]
Alpha after optimization: (2.1020774954596493+0.8j)
Beta after optimization: (-0.6544573282321532+0.8j)
Norm after manual normalization: 0.9999999999999661
counts after the final state with optimization is {np.str_('00001111'): np.int64(145), np.str_('00111100'): np.int64(155), np.str_('01100110'): np.int64(174), np.str_('10011001'): np.int64(23), np.str_('11000011'): np.int64(159), np.str_('11110000'): np.int64(344)}
The adapt iteration now is 11
The expectation value of a⁺(0) a(4) is 0.115885184640593
The expectation value of a⁺(0) a(6) is 9.819388940549828e-19
The expectation value of a⁺(1) a(5) is 0.11255793475412518
The expectation value of a⁺(1) a(7) is 2.4990535004119625e-17
The expectation value of a⁺(2) a(4) is 6.276797725828776e-17
The expectation value of a⁺(2) a(6) is 0.11563118873340474
The expectation value of a⁺(3) a(5) is 7.565043522415418e-18
The expectation value of a⁺(3) a(7) is 0.11234919788273814
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.044007358573654995
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 1.072819839161149e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 5.3532953384316205e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.004401760305974636
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.010575763602052025
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 5.151019739831071e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.027823885365871326
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.050388779966281105
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 5.367727559092413e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 4.6257229968233614e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.21329553363601592
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.03026716274902109
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 8.261485827167498e-19
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.01504139280536742
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.0075896250397598265
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 4.646218820922121e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 5.258746059630925e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.04429646653855726
The highest operator value is 0.21329553363601592 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 12
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

───────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.866066775668121
        x: [ 1.51652926+0.j  -1.61540306+0.j  ...  1.56643613+0.8j
            -0.47817892+0.8j]
      nit: 24
      jac: [-6.407e-07  7.749e-07 ...  4.768e-07  1.431e-06]
 hess_inv: [[ 3.055e+00 -8.778e-01 ... -3.718e-01  1.266e+00]
            [-8.778e-01  5.986e+00 ... -7.860e-01 -2.320e+00]
            ...
            [-3.718e-01 -7.860e-01 ...  1.626e+00  1.685e-01]
            [ 1.266e+00 -2.320e+00 ...  1.685e-01  4.124e+00]]
     nfev: 594
     njev: 35
Params1 after optimization: [ 1.51652926+0.j -1.61540306+0.j  1.88279435+0.j  0.183838  +0.j
  0.18383563+0.j  0.18383625+0.j  0.18383834+0.j  0.18383876+0.j
  0.18383818+0.j  0.18383796+0.j  0.18383739+0.j  0.18384007+0.j]
Params2 after optimization: [2.19135061+0.j]
Alpha after optimization: (1.5664361276707617+0.8j)
Beta after optimization: (-0.47817891624359304+0.8j)
Norm after manual normalization: 0.9999999999999618
counts after the final state with optimization is {np.str_('00001111'): np.int64(140), np.str_('00111100'): np.int64(167), np.str_('01100110'): np.int64(166), np.str_('10011001'): np.int64(13), np.str_('11000011'): np.int64(179), np.str_('11110000'): np.int64(335)}
The adapt iteration now is 12
The expectation value of a⁺(0) a(4) is 0.11578200736473984
The expectation value of a⁺(0) a(6) is 8.073023233717555e-18
The expectation value of a⁺(1) a(5) is 0.11244936774238182
The expectation value of a⁺(1) a(7) is 1.106510386229917e-17
The expectation value of a⁺(2) a(4) is 5.2094885402157285e-17
The expectation value of a⁺(2) a(6) is 0.11551786689072344
The expectation value of a⁺(3) a(5) is 4.163195276097804e-19
The expectation value of a⁺(3) a(7) is 0.11223198267471436
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.048423104015323265
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 9.640302229822311e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.7765715941983702e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.005323737823774887
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.01048133899396117
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.524769492582192e-17
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.028835385711536828
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.041338149439048705
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.743193931793031e-17
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 5.796155996435209e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.2120821604943703
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.031189809965691925
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 9.285244347191621e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.014784569473996029
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.008658481668202735
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 6.142517196256696e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.5526694360488257e-17
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.04874770750685936
The highest operator value is 0.2120821604943703 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 13
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756396698
        x: [ 1.6103348 +0.j  -1.4175616 +0.j  ...  1.271662  +0.8j
            -0.45681295+0.8j]
      nit: 25
      jac: [ 3.234e-06 -5.960e-08 ... -5.811e-07 -1.565e-06]
 hess_inv: [[ 3.262e+00 -1.074e+00 ...  5.763e-01  6.686e-01]
            [-1.074e+00  5.714e+00 ... -2.144e+00 -1.236e+00]
            ...
            [ 5.763e-01 -2.144e+00 ...  1.764e+00  1.002e+00]
            [ 6.686e-01 -1.236e+00 ...  1.002e+00  2.125e+00]]
     nfev: 665
     njev: 37
Params1 after optimization: [ 1.6103348 +0.j -1.4175616 +0.j  1.8827836 +0.j  0.16545274+0.j
  0.16545308+0.j  0.16545687+0.j  0.16545654+0.j  0.16545415+0.j
  0.1654509 +0.j  0.16545414+0.j  0.1654542 +0.j  0.16545543+0.j
  0.16545552+0.j]
Params2 after optimization: [2.72936826+0.j]
Alpha after optimization: (1.2716619950196786+0.8j)
Beta after optimization: (-0.4568129514449707+0.8j)
Norm after manual normalization: 0.9999999999999607
counts after the final state with optimization is {np.str_('00001111'): np.int64(158), np.str_('00111100'): np.int64(173), np.str_('01100110'): np.int64(211), np.str_('10011001'): np.int64(2), np.str_('11000011'): np.int64(182), np.str_('11110000'): np.int64(274)}
The adapt iteration now is 13
The expectation value of a⁺(0) a(4) is 0.11568152820640758
The expectation value of a⁺(0) a(6) is 4.22361095312212e-17
The expectation value of a⁺(1) a(5) is 0.11266074320027966
The expectation value of a⁺(1) a(7) is 9.902003583463318e-17
The expectation value of a⁺(2) a(4) is 1.2135957707740254e-17
The expectation value of a⁺(2) a(6) is 0.11549627354697971
The expectation value of a⁺(3) a(5) is 3.415184736019807e-18
The expectation value of a⁺(3) a(7) is 0.11250623433916944
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.017725727566899385
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 1.1475479130730208e-17
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 2.0010775531306793e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 4.259305576400502e-06
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.009786726255275305
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.321840342345176e-17
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.02088188080222541
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.10678659175681951
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.964185196051499e-17
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 1.3434349129046727e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.21880248404841124
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.02384825944963965
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 1.1230550152805045e-17
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.015208438586156198
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.002272361806445612
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 1.920547055210408e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.3391002749436996e-17
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.01777103492344774
The highest operator value is 0.21880248404841124 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 14
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

─────────────────────────────────────────────────────────────────────────────────────────────────────┤
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──│────────────────────────────────│────────────────────────────────│────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤

   State
   State
   State
   State
   State
   State
   State
   State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.78539816+0.j  0.6       +0.8j
 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667748552837
        x: [ 1.66078114+0.j  -1.29205037+0.j  ...  2.23075494+0.8j
            -0.89051333+0.8j]
      nit: 24
      jac: [ 3.368e-06 -3.666e-06 ...  2.801e-06  6.974e-06]
 hess_inv: [[ 3.917e+00 -1.341e+00 ...  3.123e+00  2.512e-01]
            [-1.341e+00  4.508e+00 ... -3.173e+00 -2.412e-01]
            ...
            [ 3.123e+00 -3.173e+00 ...  8.262e+00  2.854e+00]
            [ 2.512e-01 -2.412e-01 ...  2.854e+00  3.706e+00]]
     nfev: 588
     njev: 31
Params1 after optimization: [ 1.66078114+0.j -1.29205037+0.j  1.88279231+0.j  0.15040635+0.j
  0.15040777+0.j  0.1504105 +0.j  0.15040684+0.j  0.15040724+0.j
  0.15040475+0.j  0.15040606+0.j  0.15040712+0.j  0.15041138+0.j
  0.15040465+0.j  0.15040958+0.j]
Params2 after optimization: [2.97000721+0.j]
Alpha after optimization: (2.2307549423162008+0.8j)
Beta after optimization: (-0.8905133266741843+0.8j)
Norm after manual normalization: 0.9999999999999554
counts after the final state with optimization is {np.str_('00001111'): np.int64(154), np.str_('00111100'): np.int64(203), np.str_('01100110'): np.int64(190), np.str_('10011001'): np.int64(1), np.str_('11000011'): np.int64(183), np.str_('11110000'): np.int64(269)}
The adapt iteration now is 14
The expectation value of a⁺(0) a(4) is 0.11470854721421025
The expectation value of a⁺(0) a(6) is 1.3462555078764745e-17
The expectation value of a⁺(1) a(5) is 0.11216869935386253
The expectation value of a⁺(1) a(7) is 1.721327986667846e-16
The expectation value of a⁺(2) a(4) is 1.8883495764740739e-16
The expectation value of a⁺(2) a(6) is 0.11457471385298162
The expectation value of a⁺(3) a(5) is 1.9295168181053878e-19
The expectation value of a⁺(3) a(7) is 0.11205023515468684
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.001163965470727668
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 6.876295268258497e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 4.2135116864184774e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.0016454064755858375
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.00787195815456826
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 3.2693097910308877e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.015539947194481318
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.14489315774938508
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 3.728122990517675e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 3.518605283293055e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.22038028844954927
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.01882850928971183
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 6.874351854163518e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.013882531915311658
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 1.8510177511674858e-05
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 3.152662698962536e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 3.4479133089995115e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.0010391067078900085
The highest operator value is 0.22038028844954927 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 15
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

───────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.78539816+0.j
 0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756698026
        x: [ 1.17131313+0.j  -2.13112376+0.j  ...  2.05238765+0.8j
             0.69519136+0.8j]
      nit: 32
      jac: [ 4.619e-07 -3.576e-07 ... -2.384e-07  7.004e-07]
 hess_inv: [[ 2.388e+00 -6.003e-01 ... -2.479e+00 -1.912e+00]
            [-6.003e-01  2.081e+00 ...  1.323e+00  1.405e+00]
            ...
            [-2.479e+00  1.323e+00 ...  1.423e+01  1.166e+01]
            [-1.912e+00  1.405e+00 ...  1.166e+01  1.015e+01]]
     nfev: 819
     njev: 41
Params1 after optimization: [ 1.17131313+0.j -2.13112376+0.j  1.88279216+0.j  0.13787544+0.j
  0.13787113+0.j  0.13787978+0.j  0.13787926+0.j  0.13787464+0.j
  0.13787962+0.j  0.13787937+0.j  0.13787741+0.j  0.13788075+0.j
  0.13787912+0.j  0.13787855+0.j  0.13788444+0.j]
Params2 after optimization: [6.34266162+0.j]
Alpha after optimization: (2.052387646641767+0.8j)
Beta after optimization: (0.6951913581779746+0.8j)
Norm after manual normalization: 0.9999999999999526
counts after the final state with optimization is {np.str_('00001111'): np.int64(68), np.str_('00111100'): np.int64(128), np.str_('01100110'): np.int64(112), np.str_('10011001'): np.int64(97), np.str_('11000011'): np.int64(107), np.str_('11110000'): np.int64(488)}
The adapt iteration now is 15
The expectation value of a⁺(0) a(4) is 0.10374902668625496
The expectation value of a⁺(0) a(6) is 1.1267949661077306e-18
The expectation value of a⁺(1) a(5) is 0.10351577792769284
The expectation value of a⁺(1) a(7) is 1.2586490928445116e-17
The expectation value of a⁺(2) a(4) is 3.1991041812645764e-18
The expectation value of a⁺(2) a(6) is 0.10332695107749873
The expectation value of a⁺(3) a(5) is 1.638157709389524e-17
The expectation value of a⁺(3) a(7) is 0.10309804171169516
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.15679408995441507
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 4.471539463426195e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 3.3944058052825557e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.0407004387278662
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.007738266409005158
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.7787800603795152e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.042827845685029696
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.15203003962457085
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 3.1794035254593286e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 2.2641479078747865e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.16123803440063228
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.04291859915655822
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 4.482780582010477e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.007572394389906752
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.04721296089603379
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 1.9935629466904566e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.7189190335470897e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.15799807687978698
The highest operator value is 0.16123803440063228 for operator a⁺(1) a⁺(2) a(4) a(7)
Highest gradient excitation is [1, 2, 4, 7]
Len of ashs: 16
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[1, 2, 4, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X─╭G²(0.79)─┤  State
2: ──X─├G²(0.79)─┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: -1.6891344585930594
        x: [ 1.13850495e-03+0.j  -3.13480458e+00+0.j  ...
             1.36313852e-01+0.8j  1.36214733e-01+0.8j]
      nit: 63
      jac: [ 7.963e+00 -1.111e+01 ... -4.181e+02  4.189e+02]
 hess_inv: [[ 6.832e-04  5.031e-03 ...  3.529e-03  3.467e-03]
            [ 5.031e-03  3.969e-02 ...  2.609e-02  2.561e-02]
            ...
            [ 3.529e-03  2.609e-02 ...  3.531e-01  3.523e-01]
            [ 3.467e-03  2.561e-02 ...  3.523e-01  3.516e-01]]
     nfev: 4129
     njev: 196
Params1 after optimization: [ 1.13850495e-03+0.j -3.13480458e+00+0.j  2.97913331e-01+0.j
  2.13436749e-01+0.j  2.27685170e-01+0.j  1.93084579e-01+0.j
  2.18758965e-01+0.j  1.97195626e-01+0.j  2.14153560e-01+0.j
  2.41583334e-01+0.j  2.07049894e-01+0.j  2.17913867e-01+0.j
  1.84354788e-01+0.j  2.29175124e-01+0.j  2.07920391e-01+0.j
  2.22156514e-01+0.j]
Params2 after optimization: [5.92380409+0.j]
Alpha after optimization: (0.13631385232314602+0.8j)
Beta after optimization: (0.13621473274543303+0.8j)
Norm after manual normalization: 0.9999999999999469
counts after the final state with optimization is {np.str_('10011001'): np.int64(27), np.str_('11110000'): np.int64(973)}
The adapt iteration now is 16
The expectation value of a⁺(0) a(4) is 0.004193886456878407
The expectation value of a⁺(0) a(6) is 7.167009884143643e-17
The expectation value of a⁺(1) a(5) is 0.014545131818932466
The expectation value of a⁺(1) a(7) is 1.7035153987490482e-16
The expectation value of a⁺(2) a(4) is 5.896194730694583e-17
The expectation value of a⁺(2) a(6) is 0.00419545142149042
The expectation value of a⁺(3) a(5) is 1.9999397133196841e-16
The expectation value of a⁺(3) a(7) is 0.014317897084587871
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.3602356384695303
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 7.52779842414566e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.4040003591584453e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.290136196891298
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.07936050695937596
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 2.5970942008657957e-17
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.23430437035103474
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.27769900928420277
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.4070076317816165e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 5.920846337614385e-19
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.22201311019225756
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.23271604718669756
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 7.516326777207456e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.08226351844545841
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.3060774930552302
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 7.679671842008266e-19
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 2.6393980597270258e-17
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.3643701329358756
The highest operator value is 0.3643701329358756 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 17
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

─────────────────────────────────────────────────────────────────────────────────────────────────────┤
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)──────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─┤
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─┤
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─┤

   State
   State
   State
   State
   State
   State
   State
   State
Latest excitation: [[2, 3, 6, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X───────────┤  State
2: ──X─╭G²(0.79)─┤  State
3: ──X─├G²(0.79)─┤  State
6: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667754174185
        x: [-1.28782009+0.j  -0.34543771+0.j  ...  0.84182074+0.8j
             0.85365541+0.8j]
      nit: 21
      jac: [-8.464e-06  2.235e-07 ... -6.944e-06  6.914e-06]
 hess_inv: [[ 4.071e+00 -8.327e-02 ... -5.517e-01 -4.405e-02]
            [-8.327e-02  1.234e+00 ...  2.330e-02  1.580e-01]
            ...
            [-5.517e-01  2.330e-02 ...  8.761e-01  3.602e-01]
            [-4.405e-02  1.580e-01 ...  3.602e-01  8.493e-01]]
     nfev: 571
     njev: 26
Params1 after optimization: [-1.28782009+0.j -0.34543771+0.j  1.78465393+0.j -0.27162596+0.j
 -0.27162599+0.j -0.27162588+0.j -0.27162494+0.j -0.27162545+0.j
 -0.27162603+0.j -0.27162669+0.j -0.27162529+0.j -0.27162542+0.j
 -0.27162529+0.j -0.27162445+0.j -0.27162621+0.j -0.27162592+0.j
  1.20287519+0.j]
Params2 after optimization: [2.16150204+0.j]
Alpha after optimization: (0.8418207439058876+0.8j)
Beta after optimization: (0.8536554118059775+0.8j)
Norm after manual normalization: 0.9999999999999477
counts after the final state with optimization is {np.str_('00001111'): np.int64(171), np.str_('00111100'): np.int64(182), np.str_('01100110'): np.int64(208), np.str_('10011001'): np.int64(221), np.str_('11000011'): np.int64(218)}
The adapt iteration now is 17
The expectation value of a⁺(0) a(4) is 0.04179159894317952
The expectation value of a⁺(0) a(6) is 3.754998728529803e-17
The expectation value of a⁺(1) a(5) is 0.041793836261635076
The expectation value of a⁺(1) a(7) is 1.0328765398072186e-18
The expectation value of a⁺(2) a(4) is 5.0502055554290336e-17
The expectation value of a⁺(2) a(6) is 0.07309785984500766
The expectation value of a⁺(3) a(5) is 4.310342756690074e-17
The expectation value of a⁺(3) a(7) is 0.0730970193845412
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.44059320153621995
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 2.0387226424402487e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 9.8456109350107e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.0823889392786188
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.11061198033694782
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 7.25719768353594e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.07872373564201972
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.04872008686580209
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.0603197670339597e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 2.0347693377163493e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.04870454195189423
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.07872385361665843
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 1.935250186260659e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.11061219596166384
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.08171875379048733
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 1.857710679068157e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 3.973918455344505e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.06423522007337767
The highest operator value is 0.44059320153621995 for operator a⁺(0) a⁺(1) a(4) a(5)
Highest gradient excitation is [0, 1, 4, 5]
Len of ashs: 18
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─────────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────┤  State
──│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─┤  State
───────────────────────────────────┤  State
───────────────────────────────────┤  State
Latest excitation: [[0, 1, 4, 5]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X─╭G²(0.79)─┤  State
1: ──X─├G²(0.79)─┤  State
2: ──X─│─────────┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
5: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.78539816+0.j  0.6       +0.8j
 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.866066775646193
        x: [ 1.80560093+0.j  -0.42629805+0.j  ...  0.86632647+0.8j
             0.7109679 +0.8j]
      nit: 22
      jac: [-9.537e-07  1.565e-06 ... -5.782e-06  7.018e-06]
 hess_inv: [[ 4.607e+00  2.858e-01 ...  2.399e-01 -2.548e-01]
            [ 2.858e-01  1.269e+00 ... -2.809e-02 -1.443e-01]
            ...
            [ 2.399e-01 -2.809e-02 ...  8.780e-01  2.133e-01]
            [-2.548e-01 -1.443e-01 ...  2.133e-01  8.617e-01]]
     nfev: 597
     njev: 26
Params1 after optimization: [ 1.80560093+0.j -0.42629805+0.j -1.41078912+0.j -0.18757343+0.j
 -0.18757361+0.j -0.18757342+0.j -0.18757291+0.j -0.18757336+0.j
 -0.18757255+0.j -0.18757309+0.j -0.18757306+0.j -0.18757242+0.j
 -0.18757265+0.j -0.18757276+0.j -0.18757317+0.j -0.18757174+0.j
  0.83479031+0.j  2.53298046+0.j]
Params2 after optimization: [2.3675177+0.j]
Alpha after optimization: (0.8663264727868082+0.8j)
Beta after optimization: (0.7109678997810324+0.8j)
Norm after manual normalization: 0.999999999999943
counts after the final state with optimization is {np.str_('00001111'): np.int64(181), np.str_('00111100'): np.int64(82), np.str_('01100110'): np.int64(232), np.str_('10011001'): np.int64(230), np.str_('11000011'): np.int64(227), np.str_('11110000'): np.int64(48)}
The adapt iteration now is 18
The expectation value of a⁺(0) a(4) is 0.08163171456961535
The expectation value of a⁺(0) a(6) is 1.7544687221965467e-17
The expectation value of a⁺(1) a(5) is 0.08163174204447746
The expectation value of a⁺(1) a(7) is 4.2591638452136205e-17
The expectation value of a⁺(2) a(4) is 2.876292696152876e-17
The expectation value of a⁺(2) a(6) is 0.06842717288667269
The expectation value of a⁺(3) a(5) is 1.2400865140927266e-17
The expectation value of a⁺(3) a(7) is 0.0684271229802964
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.1578711257316666
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 3.666728606608705e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.2485887376620905e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.04354618486260835
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.05507850071336927
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 7.538593172266335e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.03146246663142247
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.07670883425459508
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 7.15547348816996e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 2.551395513311989e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.0767094478285991
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.031462461464164426
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 3.559834032502476e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.05507849126906368
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.05044182255205661
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 2.759545749312967e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 9.74884686905634e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.41225099784372005
The highest operator value is 0.41225099784372005 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 19
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─────────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)──────────────────────────────────┤  State
──│────────────────────────────────╭FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────┤  State
──╰FermionicDoubleExcitation(0.00)─│────────────────────────────────┤  State
───────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[2, 3, 6, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X───────────┤  State
2: ──X─╭G²(0.79)─┤  State
3: ──X─├G²(0.79)─┤  State
6: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.78539816+0.j
 0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756523364
        x: [-1.10303564+0.j  -0.4717012 +0.j  ...  0.67662206+0.8j
             0.72115177+0.8j]
      nit: 20
      jac: [ 4.619e-07 -1.207e-06 ...  7.287e-06 -6.914e-06]
 hess_inv: [[ 4.015e+00  3.812e-01 ... -3.643e-01 -8.327e-02]
            [ 3.812e-01  1.408e+00 ...  1.005e-01 -1.307e-01]
            ...
            [-3.643e-01  1.005e-01 ...  7.136e-01  3.161e-01]
            [-8.327e-02 -1.307e-01 ...  3.161e-01  7.004e-01]]
     nfev: 575
     njev: 24
Params1 after optimization: [-1.10303564+0.j -0.4717012 +0.j  2.06319993+0.j -0.22287202+0.j
 -0.22287122+0.j -0.22287201+0.j -0.22287203+0.j -0.22287215+0.j
 -0.2228721 +0.j -0.22287189+0.j -0.22287175+0.j -0.22287094+0.j
 -0.22287183+0.j -0.22287186+0.j -0.22287217+0.j -0.22287131+0.j
  1.02352908+0.j  0.88095771+0.j  1.02352935+0.j]
Params2 after optimization: [2.11058629+0.j]
Alpha after optimization: (0.6766220621665355+0.8j)
Beta after optimization: (0.7211517743182793+0.8j)
Norm after manual normalization: 0.999999999999939
counts after the final state with optimization is {np.str_('00001111'): np.int64(160), np.str_('00111100'): np.int64(216), np.str_('01100110'): np.int64(202), np.str_('10011001'): np.int64(193), np.str_('11000011'): np.int64(227), np.str_('11110000'): np.int64(2)}
The adapt iteration now is 19
The expectation value of a⁺(0) a(4) is 0.0333633530648908
The expectation value of a⁺(0) a(6) is 8.534708998463451e-17
The expectation value of a⁺(1) a(5) is 0.033363319729093115
The expectation value of a⁺(1) a(7) is 3.556419559956289e-18
The expectation value of a⁺(2) a(4) is 4.7312383501122225e-17
The expectation value of a⁺(2) a(6) is 0.07088247303305685
The expectation value of a⁺(3) a(5) is 9.658138537546476e-17
The expectation value of a⁺(3) a(7) is 0.07088248438426256
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.42949069044317933
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 2.1574429099755584e-19
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 6.5726127315635644e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.09139063586912954
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.12820030193013987
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 2.7136347234490238e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.09615129605169846
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.03098322172982758
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.0164306516842835e-17
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 2.1913013453796974e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.030983413900308295
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.09615129466981628
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 2.77948302762853e-19
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.12820029940444488
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.09102872850509101
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 2.494845272856591e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 7.960563201305654e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.030822600380039915
The highest operator value is 0.42949069044317933 for operator a⁺(0) a⁺(1) a(4) a(5)
Highest gradient excitation is [0, 1, 4, 5]
Len of ashs: 20
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─────────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────╭FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)─┤
──│────────────────────────────────╭FermionicDoubleExcitation(0.00)─│────────────────────────────────┤
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤
──╰FermionicDoubleExcitation(0.00)─│────────────────────────────────╰FermionicDoubleExcitation(0.00)─┤
───────────────────────────────────├FermionicDoubleExcitation(0.00)──────────────────────────────────┤
───────────────────────────────────╰FermionicDoubleExcitation(0.00)──────────────────────────────────┤

   State
   State
   State
   State
   State
   State
   State
   State
Latest excitation: [[0, 1, 4, 5]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X─╭G²(0.79)─┤  State
1: ──X─├G²(0.79)─┤  State
2: ──X─│─────────┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
5: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.866066775497662
        x: [ 1.33416555+0.j  -0.74026304+0.j  ...  0.7947631 +0.8j
             0.55982066+0.8j]
      nit: 24
      jac: [-3.472e-06 -5.424e-06 ...  4.679e-06 -6.646e-06]
 hess_inv: [[ 3.381e+00 -3.301e-01 ... -4.045e-02  5.864e-01]
            [-3.301e-01  1.385e+00 ... -4.363e-02 -5.058e-02]
            ...
            [-4.045e-02 -4.363e-02 ...  8.131e-01  1.962e-01]
            [ 5.864e-01 -5.058e-02 ...  1.962e-01  7.882e-01]]
     nfev: 724
     njev: 29
Params1 after optimization: [ 1.33416555+0.j -0.74026304+0.j -2.00174979+0.j -0.17757771+0.j
 -0.17757877+0.j -0.17757916+0.j -0.17757795+0.j -0.17757922+0.j
 -0.17757901+0.j -0.17757871+0.j -0.17757803+0.j -0.17757809+0.j
 -0.17757883+0.j -0.17757882+0.j -0.17757847+0.j -0.17757967+0.j
  1.03518736+0.j  2.18816209+0.j  1.0351865 +0.j  2.18816191+0.j]
Params2 after optimization: [2.52037016+0.j]
Alpha after optimization: (0.794763101275319+0.8j)
Beta after optimization: (0.5598206571734792+0.8j)
Norm after manual normalization: 0.9999999999999372
counts after the final state with optimization is {np.str_('00001111'): np.int64(189), np.str_('00111100'): np.int64(32), np.str_('01100110'): np.int64(207), np.str_('10011001'): np.int64(253), np.str_('11000011'): np.int64(230), np.str_('11110000'): np.int64(89)}
The adapt iteration now is 20
The expectation value of a⁺(0) a(4) is 0.08648466877213637
The expectation value of a⁺(0) a(6) is 9.698305157653799e-18
The expectation value of a⁺(1) a(5) is 0.08648519037519282
The expectation value of a⁺(1) a(7) is 5.4333940451347456e-17
The expectation value of a⁺(2) a(4) is 4.1633645012670864e-17
The expectation value of a⁺(2) a(6) is 0.08123824808775729
The expectation value of a⁺(3) a(5) is 7.717336564462944e-18
The expectation value of a⁺(3) a(7) is 0.08123748397151057
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.19415935932010966
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 7.375525169925316e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.1646150806579117e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.0238126900962251
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.0289963471604213
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 2.278375270922185e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.013876241627985529
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.07010921855778755
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 8.345515395948093e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 4.328210321428207e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.07012194203118549
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.013876130780083498
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 7.452130816817941e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.028996144561391025
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.029927344732288415
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 3.874997485941034e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 3.087316674707129e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.36178654748013783
The highest operator value is 0.36178654748013783 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 21
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─────────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)
──│────────────────────────────────╭FermionicDoubleExcitation(0.00)─│───────────────────────────────
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─│────────────────────────────────╰FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─────────────────────────────────
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─────────────────────────────────

───────────────────────────────────┤  State
───────────────────────────────────┤  State
──╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────┤  State
──│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[2, 3, 6, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X───────────┤  State
2: ──X─╭G²(0.79)─┤  State
3: ──X─├G²(0.79)─┤  State
6: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756243963
        x: [-1.73639106+0.j  -0.53604981+0.j  ...  0.72764132+0.8j
             0.58968692+0.8j]
      nit: 24
      jac: [ 2.906e-06 -1.341e-07 ...  2.459e-06 -2.995e-06]
 hess_inv: [[ 7.383e+00  4.631e-01 ... -6.687e-01  7.409e-01]
            [ 4.631e-01  1.194e+00 ...  3.498e-02 -1.138e-01]
            ...
            [-6.687e-01  3.498e-02 ...  7.951e-01  2.201e-01]
            [ 7.409e-01 -1.138e-01 ...  2.201e-01  7.250e-01]]
     nfev: 753
     njev: 29
Params1 after optimization: [-1.73639106+0.j -0.53604981+0.j  1.53703367+0.j -0.17257744+0.j
 -0.17257729+0.j -0.17257723+0.j -0.17257785+0.j -0.17257727+0.j
 -0.17257749+0.j -0.17257784+0.j -0.17257755+0.j -0.17257727+0.j
 -0.17257808+0.j -0.17257772+0.j -0.17257853+0.j -0.17257778+0.j
  1.2439123 +0.j  0.95621403+0.j  1.24391273+0.j  0.95621295+0.j
  1.24391236+0.j]
Params2 after optimization: [2.37619468+0.j]
Alpha after optimization: (0.727641323786851+0.8j)
Beta after optimization: (0.5896869186571412+0.8j)
Norm after manual normalization: 0.9999999999999339
counts after the final state with optimization is {np.str_('00001111'): np.int64(216), np.str_('00111100'): np.int64(205), np.str_('01100110'): np.int64(215), np.str_('10011001'): np.int64(231), np.str_('11000011'): np.int64(76), np.str_('11110000'): np.int64(57)}
The adapt iteration now is 21
The expectation value of a⁺(0) a(4) is 0.06986555174647965
The expectation value of a⁺(0) a(6) is 3.1208135155113544e-17
The expectation value of a⁺(1) a(5) is 0.06986511696490028
The expectation value of a⁺(1) a(7) is 5.4472142619312614e-17
The expectation value of a⁺(2) a(4) is 1.350623033879204e-17
The expectation value of a⁺(2) a(6) is 0.0818297156463556
The expectation value of a⁺(3) a(5) is 5.178255517132359e-17
The expectation value of a⁺(3) a(7) is 0.08182994813477487
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.40832422886588726
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 1.170891914373242e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 9.412227825159687e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.04636142677707547
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.05284911250006716
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.5290065739353488e-17
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.02989142082657716
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.07366012874188838
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 6.931357588358821e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 4.162204169218929e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.0736654783613394
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.029891375706912374
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 1.0153540242358748e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.052849030033911615
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.04428754544986517
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 4.131099586572063e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 5.9127692086383825e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.16084968710346198
The highest operator value is 0.40832422886588726 for operator a⁺(0) a⁺(1) a(4) a(5)
Highest gradient excitation is [0, 1, 4, 5]
Len of ashs: 22
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─────────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)
──│────────────────────────────────╭FermionicDoubleExcitation(0.00)─│───────────────────────────────
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─│────────────────────────────────╰FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─────────────────────────────────
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─────────────────────────────────

───────────────────────────────────╭FermionicDoubleExcitation(0.00)─┤  State
───────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
──╭FermionicDoubleExcitation(0.00)─│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────┤  State
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────╰FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)──────────────────────────────────┤  State
──╰FermionicDoubleExcitation(0.00)──────────────────────────────────┤  State
Latest excitation: [[0, 1, 4, 5]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X─╭G²(0.79)─┤  State
1: ──X─├G²(0.79)─┤  State
2: ──X─│─────────┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
5: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.78539816+0.j  0.6       +0.8j
 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756496165
        x: [ 1.26559515+0.j  -0.57586885+0.j  ...  0.85400619+0.8j
             0.60359172+0.8j]
      nit: 27
      jac: [-4.470e-08 -5.960e-08 ...  4.351e-06 -6.095e-06]
 hess_inv: [[ 5.345e+00  1.724e-01 ... -7.915e-02  1.125e+00]
            [ 1.724e-01  1.200e+00 ... -1.081e-01  1.731e-01]
            ...
            [-7.915e-02 -1.081e-01 ...  8.457e-01  9.124e-02]
            [ 1.125e+00  1.731e-01 ...  9.124e-02  1.306e+00]]
     nfev: 890
     njev: 33
Params1 after optimization: [ 1.26559515+0.j -0.57586885+0.j -1.93556777+0.j -0.20947913+0.j
 -0.20947977+0.j -0.20948024+0.j -0.20947988+0.j -0.20948015+0.j
 -0.2094798 +0.j -0.20947928+0.j -0.2094789 +0.j -0.20947895+0.j
 -0.20948035+0.j -0.20947996+0.j -0.20947884+0.j -0.20948089+0.j
  0.87969779+0.j  1.59351311+0.j  0.87969581+0.j  1.59351271+0.j
  0.87969681+0.j  1.59351354+0.j]
Params2 after optimization: [2.5168443+0.j]
Alpha after optimization: (0.8540061929014351+0.8j)
Beta after optimization: (0.6035917244494862+0.8j)
Norm after manual normalization: 0.9999999999999302
counts after the final state with optimization is {np.str_('00001111'): np.int64(179), np.str_('00111100'): np.int64(36), np.str_('01100110'): np.int64(231), np.str_('10011001'): np.int64(209), np.str_('11000011'): np.int64(246), np.str_('11110000'): np.int64(99)}
The adapt iteration now is 22
The expectation value of a⁺(0) a(4) is 0.08638891092956938
The expectation value of a⁺(0) a(6) is 2.49611434306289e-17
The expectation value of a⁺(1) a(5) is 0.08638900788449562
The expectation value of a⁺(1) a(7) is 1.8582958493087175e-17
The expectation value of a⁺(2) a(4) is 3.434873268367682e-17
The expectation value of a⁺(2) a(6) is 0.08099723052656417
The expectation value of a⁺(3) a(5) is 1.525759653480017e-17
The expectation value of a⁺(3) a(7) is 0.08099708781133834
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.19352670445566592
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 4.592750488365901e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 3.7313434990668005e-18
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.024206711244685084
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.02948502264986151
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 1.1422604521587103e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.014170541806626087
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.07037834114069277
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 8.374217555820988e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 1.5420994782073935e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.07038070228067404
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.014170521248448871
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 4.582618935743407e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.029484985075250936
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.030343357317025803
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 1.374342043414009e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.6757462517620337e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.36301360581968906
The highest operator value is 0.36301360581968906 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 23
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─────────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)
──│────────────────────────────────╭FermionicDoubleExcitation(0.00)─│───────────────────────────────
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─│────────────────────────────────╰FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─────────────────────────────────
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─────────────────────────────────

───────────────────────────────────╭FermionicDoubleExcitation(0.00)──────────────────────────────────┤
───────────────────────────────────├FermionicDoubleExcitation(0.00)──────────────────────────────────┤
──╭FermionicDoubleExcitation(0.00)─│────────────────────────────────╭FermionicDoubleExcitation(0.00)─┤
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│────────────────────────────────┤
──│────────────────────────────────╰FermionicDoubleExcitation(0.00)─│────────────────────────────────┤
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)─┤
──╰FermionicDoubleExcitation(0.00)──────────────────────────────────╰FermionicDoubleExcitation(0.00)─┤

   State
   State
   State
   State
   State
   State
   State
   State
Latest excitation: [[2, 3, 6, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X───────────┤  State
2: ──X─╭G²(0.79)─┤  State
3: ──X─├G²(0.79)─┤  State
6: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.78539816+0.j
 0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756707183
        x: [-1.8802134 +0.j  -0.40919509+0.j  ...  0.81390726+0.8j
             0.57313049+0.8j]
      nit: 27
      jac: [ 1.192e-07 -2.980e-08 ...  0.000e+00  8.941e-08]
 hess_inv: [[ 6.023e+00  4.904e-01 ... -2.877e-01  1.666e+00]
            [ 4.904e-01  1.001e+00 ... -2.188e-02  6.967e-02]
            ...
            [-2.877e-01 -2.188e-02 ...  7.887e-01  1.877e-01]
            [ 1.666e+00  6.967e-02 ...  1.877e-01  1.250e+00]]
     nfev: 895
     njev: 32
Params1 after optimization: [-1.8802134 +0.j -0.40919509+0.j  1.28172284+0.j -0.21617613+0.j
 -0.2161764 +0.j -0.21617641+0.j -0.21617523+0.j -0.21617481+0.j
 -0.21617562+0.j -0.21617577+0.j -0.2161762 +0.j -0.21617468+0.j
 -0.21617517+0.j -0.21617576+0.j -0.21617582+0.j -0.21617598+0.j
  1.10794303+0.j  0.83934625+0.j  1.10794193+0.j  0.83934682+0.j
  1.10794209+0.j  0.8393463 +0.j  1.10794249+0.j]
Params2 after optimization: [2.51710688+0.j]
Alpha after optimization: (0.8139072591122517+0.8j)
Beta after optimization: (0.5731304916115757+0.8j)
Norm after manual normalization: 0.9999999999999314
counts after the final state with optimization is {np.str_('00001111'): np.int64(182), np.str_('00111100'): np.int64(223), np.str_('01100110'): np.int64(217), np.str_('10011001'): np.int64(238), np.str_('11000011'): np.int64(40), np.str_('11110000'): np.int64(100)}
The adapt iteration now is 23
The expectation value of a⁺(0) a(4) is 0.08155028425738539
The expectation value of a⁺(0) a(6) is 2.6137356406611973e-17
The expectation value of a⁺(1) a(5) is 0.08155027538317385
The expectation value of a⁺(1) a(7) is 1.1745594943687514e-17
The expectation value of a⁺(2) a(4) is 1.8592833538763395e-17
The expectation value of a⁺(2) a(6) is 0.08612164272546968
The expectation value of a⁺(3) a(5) is 3.155112418367425e-17
The expectation value of a⁺(3) a(7) is 0.08612164850565072
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.36136690217531636
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 5.450813976706904e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.0830822745580999e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.028436336621526646
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.029199846757764422
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 9.093439833234706e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.01402530923791916
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.06753939999405029
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.2976091549541881e-17
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 4.80385939728541e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.06753954403275032
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.014025307984883495
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 5.236545907978986e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.029199844467564982
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.025595673237663916
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 4.972024017406726e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 1.446590175528741e-17
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.19396499309407722
The highest operator value is 0.36136690217531636 for operator a⁺(0) a⁺(1) a(4) a(5)
Highest gradient excitation is [0, 1, 4, 5]
Len of ashs: 24
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─────────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)
──│────────────────────────────────╭FermionicDoubleExcitation(0.00)─│───────────────────────────────
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─│────────────────────────────────╰FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─────────────────────────────────
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─────────────────────────────────

───────────────────────────────────╭FermionicDoubleExcitation(0.00)─────────────────────────────────
───────────────────────────────────├FermionicDoubleExcitation(0.00)─────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─│────────────────────────────────╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──│────────────────────────────────╰FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)──────────────────────────────────╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────┤  State
──│────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)─┤  State
──╰FermionicDoubleExcitation(0.00)─┤  State
───────────────────────────────────┤  State
───────────────────────────────────┤  State
Latest excitation: [[0, 1, 4, 5]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X─╭G²(0.79)─┤  State
1: ──X─├G²(0.79)─┤  State
2: ──X─│─────────┤  State
3: ──X─│─────────┤  State
4: ────├G²(0.79)─┤  State
5: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756259524
        x: [ 1.30184952+0.j  -0.42705967+0.j  ...  0.8123496 +0.8j
             0.62311476+0.8j]
      nit: 28
      jac: [ 2.339e-06 -1.416e-06 ...  1.401e-06 -1.788e-06]
 hess_inv: [[ 4.762e+00  1.190e-01 ... -3.584e-01  1.315e+00]
            [ 1.190e-01  1.102e+00 ... -1.158e-01  2.026e-01]
            ...
            [-3.584e-01 -1.158e-01 ...  8.343e-01  4.859e-02]
            [ 1.315e+00  2.026e-01 ...  4.859e-02  1.527e+00]]
     nfev: 985
     njev: 34
Params1 after optimization: [ 1.30184952+0.j -0.42705967+0.j -1.906361  +0.j -0.20258218+0.j
 -0.2025825 +0.j -0.20258049+0.j -0.20258224+0.j -0.20258277+0.j
 -0.20258118+0.j -0.20258187+0.j -0.20258178+0.j -0.20258128+0.j
 -0.20258185+0.j -0.2025812 +0.j -0.20258197+0.j -0.20258338+0.j
  0.65519526+0.j  1.12730858+0.j  0.65519704+0.j  1.12730932+0.j
  0.65519655+0.j  1.12730873+0.j  0.65519635+0.j  1.12730933+0.j]
Params2 after optimization: [2.43356826+0.j]
Alpha after optimization: (0.812349604348552+0.8j)
Beta after optimization: (0.6231147550800991+0.8j)
Norm after manual normalization: 0.9999999999999272
counts after the final state with optimization is {np.str_('00001111'): np.int64(199), np.str_('00111100'): np.int64(57), np.str_('01100110'): np.int64(210), np.str_('10011001'): np.int64(240), np.str_('11000011'): np.int64(229), np.str_('11110000'): np.int64(65)}
The adapt iteration now is 24
The expectation value of a⁺(0) a(4) is 0.0839110492260176
The expectation value of a⁺(0) a(6) is 7.958034812762757e-18
The expectation value of a⁺(1) a(5) is 0.08391110350130221
The expectation value of a⁺(1) a(7) is 1.4663995028048463e-17
The expectation value of a⁺(2) a(4) is 3.0599403149173782e-18
The expectation value of a⁺(2) a(6) is 0.07462330631020986
The expectation value of a⁺(3) a(5) is 1.2969315105317112e-17
The expectation value of a⁺(3) a(7) is 0.0746232167312168
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.1761730974786177
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 5.896179442821292e-18
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 1.3020697885021953e-17
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.0342684011544367
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.042418382002760706
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 9.912309263106947e-18
The expectation value of a⁺(0) a⁺(3) a(4) a(7) is 0.02248835409255658
The expectation value of a⁺(0) a⁺(3) a(5) a(6) is 0.07525845438368764
The expectation value of a⁺(0) a⁺(3) a(6) a(7) is 1.1436842588887889e-17
The expectation value of a⁺(1) a⁺(2) a(4) a(5) is 2.7440618164008565e-18
The expectation value of a⁺(1) a⁺(2) a(4) a(7) is 0.0752597192610503
The expectation value of a⁺(1) a⁺(2) a(5) a(6) is 0.022488343258635895
The expectation value of a⁺(1) a⁺(2) a(6) a(7) is 5.7436352836271106e-18
The expectation value of a⁺(1) a⁺(3) a(5) a(7) is 0.04241836220137707
The expectation value of a⁺(2) a⁺(3) a(4) a(5) is 0.040877668136388864
The expectation value of a⁺(2) a⁺(3) a(4) a(7) is 2.972969168225612e-18
The expectation value of a⁺(2) a⁺(3) a(5) a(6) is 9.370307892982688e-18
The expectation value of a⁺(2) a⁺(3) a(6) a(7) is 0.3913549905374311
The highest operator value is 0.3913549905374311 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 25
Exc. going to prepare state1 [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7]]
0: ──X──────────────────────────────────────────────────────────────────
1: ──X──────────────────────────────────╭FermionicDoubleExcitation(0.00)
2: ──X─╭FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
3: ──X─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
4: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
5: ────│────────────────────────────────├FermionicDoubleExcitation(0.00)
6: ────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
7: ────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────────────────────────────────────
──├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

────────────────────────────────────────────────────────────────────────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)─────────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─╭FermionicDoubleExcitation(0.00)
──│────────────────────────────────│────────────────────────────────├FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)─├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)─╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)
──│────────────────────────────────╭FermionicDoubleExcitation(0.00)─│───────────────────────────────
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)─│────────────────────────────────╰FermionicDoubleExcitation(0.00)
───────────────────────────────────├FermionicDoubleExcitation(0.00)─────────────────────────────────
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─────────────────────────────────

───────────────────────────────────╭FermionicDoubleExcitation(0.00)─────────────────────────────────
───────────────────────────────────├FermionicDoubleExcitation(0.00)─────────────────────────────────
──╭FermionicDoubleExcitation(0.00)─│────────────────────────────────╭FermionicDoubleExcitation(0.00)
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────├FermionicDoubleExcitation(0.00)
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─│───────────────────────────────
──│────────────────────────────────╰FermionicDoubleExcitation(0.00)─│───────────────────────────────
──├FermionicDoubleExcitation(0.00)──────────────────────────────────├FermionicDoubleExcitation(0.00)
──╰FermionicDoubleExcitation(0.00)──────────────────────────────────╰FermionicDoubleExcitation(0.00)

──╭FermionicDoubleExcitation(0.00)──────────────────────────────────┤  State
──├FermionicDoubleExcitation(0.00)──────────────────────────────────┤  State
──│────────────────────────────────╭FermionicDoubleExcitation(0.00)─┤  State
──│────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
──├FermionicDoubleExcitation(0.00)─│────────────────────────────────┤  State
──╰FermionicDoubleExcitation(0.00)─│────────────────────────────────┤  State
───────────────────────────────────├FermionicDoubleExcitation(0.00)─┤  State
───────────────────────────────────╰FermionicDoubleExcitation(0.00)─┤  State
Latest excitation: [[2, 3, 6, 7]]
Length of latest excitation: 1
Params2 are [0.78539816]
0: ──X───────────┤  State
1: ──X───────────┤  State
2: ──X─╭G²(0.79)─┤  State
3: ──X─├G²(0.79)─┤  State
6: ────├G²(0.79)─┤  State
7: ────╰G²(0.79)─┤  State
[0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
 0.        +0.j  0.78539816+0.j  0.6       +0.8j 0.6       +0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.8660667756477731
        x: [-1.87874869+0.j  -0.33937282+0.j  ...  0.75671359+0.8j
             0.53676868+0.8j]
      nit: 29
      jac: [-9.984e-07 -9.984e-07 ... -1.132e-06  1.639e-06]
 hess_inv: [[ 6.426e+00  4.852e-01 ... -1.622e-01  2.359e+00]
            [ 4.852e-01  9.864e-01 ... -8.090e-03  1.281e-01]
            ...
            [-1.622e-01 -8.090e-03 ...  7.915e-01  1.448e-01]
            [ 2.359e+00  1.281e-01 ...  1.448e-01  1.674e+00]]
     nfev: 1019
     njev: 34
Params1 after optimization: [-1.87874869+0.j -0.33937282+0.j  1.27650795+0.j -0.22073254+0.j
 -0.22073091+0.j -0.22073248+0.j -0.22073035+0.j -0.22073181+0.j
 -0.22073372+0.j -0.220731  +0.j -0.22073184+0.j -0.22073083+0.j
 -0.22073124+0.j -0.22073127+0.j -0.22073297+0.j -0.22073159+0.j
  0.88971303+0.j  0.64696727+0.j  0.8897127 +0.j  0.64696718+0.j
  0.88971299+0.j  0.6469664 +0.j  0.8897129 +0.j  0.64696786+0.j
  0.8897134 +0.j]
Params2 after optimization: [2.50946967+0.j]
Alpha after optimization: (0.7567135900383853+0.8j)
Beta after optimization: (0.5367686759233717+0.8j)
Norm after manual normalization: 0.9999999999999242
counts after the final state with optimization is {np.str_('00001111'): np.int64(186), np.str_('00111100'): np.int64(225), np.str_('01100110'): np.int64(208), np.str_('10011001'): np.int64(244), np.str_('11000011'): np.int64(42), np.str_('11110000'): np.int64(95)}
The params after GS is [-1.87874869+0.j -0.33937282+0.j  1.27650795+0.j -0.22073254+0.j
 -0.22073091+0.j -0.22073248+0.j -0.22073035+0.j -0.22073181+0.j
 -0.22073372+0.j -0.220731  +0.j -0.22073184+0.j -0.22073083+0.j
 -0.22073124+0.j -0.22073127+0.j -0.22073297+0.j -0.22073159+0.j
  0.88971303+0.j  0.64696727+0.j  0.8897127 +0.j  0.64696718+0.j
  0.88971299+0.j  0.6469664 +0.j  0.8897129 +0.j  0.64696786+0.j
  0.8897134 +0.j]
Ash excitation after gs state: [[2, 3, 6, 7], [1, 2, 4, 7], [0, 1, 4, 5], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [1, 2, 4, 7], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7], [0, 1, 4, 5], [2, 3, 6, 7]]
Energies: [np.float64(-1.4300836706628741), np.float64(-1.5694603835508567), np.float64(-1.7440626583927687), np.float64(-1.8660667755901288), np.float64(-1.8660667756515545), np.float64(-1.8660667755524885), np.float64(-1.8660667756599194), np.float64(-1.8660667755164622), np.float64(-1.866066775450773), np.float64(-1.866066775578696), np.float64(-1.8660667756202018), np.float64(-1.866066775668121), np.float64(-1.8660667756396698), np.float64(-1.8660667748552837), np.float64(-1.8660667756698026), np.float64(-1.6891344585930594), np.float64(-1.8660667754174185), np.float64(-1.866066775646193), np.float64(-1.8660667756523364), np.float64(-1.866066775497662), np.float64(-1.8660667756243963), np.float64(-1.8660667756496165), np.float64(-1.8660667756707183), np.float64(-1.8660667756259524), np.float64(-1.8660667756477731)]
Old gradient: [np.float64(0.3100451588582134), np.float64(0.3467832177041655), np.float64(0.426790864300441), np.float64(0.36630045661318256), np.float64(0.1637674731049467), np.float64(0.16562011704197205), np.float64(0.1788165646883157), np.float64(0.1825860536823775), np.float64(0.1921376697111764), np.float64(0.20229430833407355), np.float64(0.1982043729227405), np.float64(0.21329553363601592), np.float64(0.2120821604943703), np.float64(0.21880248404841124), np.float64(0.22038028844954927), np.float64(0.16123803440063228), np.float64(0.3643701329358756), np.float64(0.44059320153621995), np.float64(0.41225099784372005), np.float64(0.42949069044317933), np.float64(0.36178654748013783), np.float64(0.40832422886588726), np.float64(0.36301360581968906), np.float64(0.36136690217531636), np.float64(0.3913549905374311)]


In [6]:
energies_rep_c2 = [np.float64(-1.4300836706628741), np.float64(-1.5694603835508567), np.float64(-1.7440626583927687), np.float64(-1.8660667755901288), np.float64(-1.8660667756515545), np.float64(-1.8660667755524885), np.float64(-1.8660667756599194), np.float64(-1.8660667755164622), np.float64(-1.866066775450773), np.float64(-1.866066775578696), np.float64(-1.8660667756202018), np.float64(-1.866066775668121), np.float64(-1.8660667756396698), np.float64(-1.8660667748552837), np.float64(-1.8660667756698026), np.float64(-1.6891344585930594), np.float64(-1.8660667754174185), np.float64(-1.866066775646193), np.float64(-1.8660667756523364), np.float64(-1.866066775497662), np.float64(-1.8660667756243963), np.float64(-1.8660667756496165), np.float64(-1.8660667756707183), np.float64(-1.8660667756259524), np.float64(-1.8660667756477731)]
grad_rep_c2 =  [np.float64(0.3100451588582134), np.float64(0.3467832177041655), np.float64(0.426790864300441), np.float64(0.36630045661318256), np.float64(0.1637674731049467), np.float64(0.16562011704197205), np.float64(0.1788165646883157), np.float64(0.1825860536823775), np.float64(0.1921376697111764), np.float64(0.20229430833407355), np.float64(0.1982043729227405), np.float64(0.21329553363601592), np.float64(0.2120821604943703), np.float64(0.21880248404841124), np.float64(0.22038028844954927), np.float64(0.16123803440063228), np.float64(0.3643701329358756), np.float64(0.44059320153621995), np.float64(0.41225099784372005), np.float64(0.42949069044317933), np.float64(0.36178654748013783), np.float64(0.40832422886588726), np.float64(0.36301360581968906), np.float64(0.36136690217531636), np.float64(0.3913549905374311)]


In [8]:
len(grad_rep_c2)

25